## End-to-End Procedure

### Procedure Outline
1. Normalize the dataset
    - Detect faces among all the images. Reject images that have more than one face.
    - Crop the face from the image.
    - Align it.
2. Generate Train-Test Splits
    - Create folds.
3. Evaluate 
    - Generate embeddings from the splits
    - Train classifier on the embeddings
    - Test classifier on the embeddings
4. Tune classifier
    - Tune the classifier 
5. Save the model

### Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import cv2
import pprint
import logging
import tqdm

In [3]:
import face_trigger

from face_trigger.model.deep.FaceRecognizer import FaceRecognizer
from face_trigger.process.post_process import FaceDetector, LandmarkDetector, FaceAlign
from face_trigger.utils.common import RepeatedTimer, clamp_rectangle
from face_trigger.utils.Dataset import Dataset

In [4]:
unnormalized_dataset_path = "/media/ankurrc/new_volume/softura/facerec/datasets/standard_att"
dataset_path = "/media/ankurrc/new_volume/softura/facerec/att_norm"
split_path = "/media/ankurrc/new_volume/softura/facerec/att_split_path"

In [5]:
logging.basicConfig(level=logging.DEBUG)

### Normalize dataset
 While normalizing the dataset we assume that the original dataset has the following structure:
 1. At the root level there are directories that represent each personality. The directories may or may not have a numeric name.
 2. Within each directory, the files should represent the images that contain the parent directory's(personality) faces. The file names may or may not be numeric.
 
 The final dimensions is assumed to be 256x256, since that is what the DNN ingests.
 Also, the detected faces align the eyes about 0.35th of the width from the ends. 

In [4]:
import uuid

In [53]:
def normalize_dataset(dataset_path=None, output_path=None):

    logger = logging.getLogger(__name__)

    face_detector = FaceDetector()
    face_align = FaceAlign(left_eye_offset=(0.35, 0.35), final_width=256)
    landmark_predictor = LandmarkDetector()

    rejected_faces = {}

    bar = tqdm.tqdm(total=None)

    if not os.path.exists(dataset_path):
        raise Exception("Invalid dataset path!")

    # setup output directory
    if os.path.isdir(output_path):
        os.rename(output_path, os.path.join(os.path.split(
            output_path)[0], os.path.split(
            output_path)[1] + "_" + str(uuid.uuid4().get_hex())))
    os.makedirs(output_path)

    for root, dirs, files in os.walk(dataset_path):

        if root == dataset_path:
            bar.total = len(dirs)

        for direc in dirs:
            # create output directory for this presonality
            output_direc_path = os.path.join(output_path, direc)
            os.mkdir(output_direc_path)

        for img in files:

            img_path = os.path.join(dataset_path, root, img)

            # read the image
            rgbImg = cv2.imread(img_path)

            grayImg = None
            if rgbImg is None:
                break
            elif rgbImg.shape[2] == 3:
                grayImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2GRAY)
            else:
                grayImg = rgbImg

            # detect faces
            faces = face_detector.detect_unbounded(grayImg)

            if len(faces) == 1:

                # get face (only interested if there's one and only one)
                face_bb = faces[0]

                # get the landmarks
                landmarks = landmark_predictor.predict(face_bb, grayImg)

                # align the face
                aligned_face = face_align.align(grayImg, landmarks)

                # write to output directory
                save_path = os.path.join(
                    output_path, os.path.basename(root), img)

                cv2.imwrite(save_path, aligned_face)

            else:
                root = os.path.basename(root)
                if root in rejected_faces:
                    rejected_faces[root].append(img)
                else:
                    rejected_faces[root] = [img]

        if root != dataset_path:
            bar.update()

    bar.close()
    logger.info("Normalized dataset created at {}".format(output_path))

    print("Rejected directories:")
    pprint.pprint(rejected_faces)

    return rejected_faces

In [54]:
rejected_dirs = normalize_dataset(
    dataset_path=unnormalized_dataset_path, output_path=dataset_path)

100%|██████████| 40/40 [00:03<00:00, 12.90it/s]
INFO:__main__:Normalized dataset created at /media/ankurrc/new_volume/softura/facerec/att_norm


Rejected directories:
{'33': ['4.png'], '35': ['2.png'], '37': ['4.png', '5.png']}


### Generate Splits

In [8]:
def generate_splits(dataset_path=None, split_path=None):
    dataset = Dataset(dataset_path=dataset_path,
                      split_path=split_path)
    folds = 3
    training_samples = [2, 5, 8]
    
    dataset.split(num_train_list=training_samples, folds=folds)

In [9]:
generate_splits(dataset_path=dataset_path, split_path=split_path)

INFO:face_trigger.utils.Dataset:Generating for 2 training samples per subject.
Dir-->20:  30%|███       | 12/40 [00:00<00:00, 100.21it/s]


Generating: Fold 1
Creating directory: /media/ankurrc/new_volume/softura/facerec/att_split_path/2/1
done.
/media/ankurrc/new_volume/softura/facerec/att_split_path/2/1/train.csv


Dir-->25:  42%|████▎     | 17/40 [00:00<00:00, 112.87it/s]

Generating: Fold 2
Creating directory: /media/ankurrc/new_volume/softura/facerec/att_split_path/2/2
done.
/media/ankurrc/new_volume/softura/facerec/att_split_path/2/2/train.csv



Dir-->20:  30%|███       | 12/40 [00:00<00:00, 88.09it/s]

...


Dir-->25:  42%|████▎     | 17/40 [00:00<00:00, 124.94it/s]

Generating: Fold 2
Creating directory: /media/ankurrc/new_volume/softura/facerec/att_split_path/8/2
done.
/media/ankurrc/new_volume/softura/facerec/att_split_path/8/2/train.csv



Dir-->23:  38%|███▊      | 15/40 [00:00<00:00, 98.00it/s]


Generating: Fold 3
Creating directory: /media/ankurrc/new_volume/softura/facerec/att_split_path/8/3
done.
/media/ankurrc/new_volume/softura/facerec/att_split_path/8/3/train.csv


100%|██████████| 3/3 [00:01<00:00,  2.40it/s] 109.07it/s]
INFO:face_trigger.utils.Dataset:The following directories were rejected: ['37']
INFO:face_trigger.utils.Dataset:We have 39 subjects in our dataset.
